# Análisis exploratorio de datos

## 1. Introducción 

El conjunto de datos que se pretende analizar en este notebook se denomina **EXIST** y representa un problema de **detección y clasificación de sexismo** a partir de conjuntos de **textos en español e inglés** procedentes de las redes sociales **Twitter y Gab**. Esta temática surge en un formato competitivo teniendo lugar su primera edición en el año 2021 con una segunda recientemente terminada en el presente año 2022.

En este notebook se detallan los aspectos más relevantes acerca de la **composición de los conjuntos de entrenamiento y validación, así como análisis estadísticos y visuales** que ayuden a comprender tanto la temática como los datos disponibles para abordarla. Para ello se ha hecho uso de los artículos [*Overview of EXIST 2021*](http://diposit.ub.edu/dspace/bitstream/2445/181257/1/715155.pdf) y [*Overview of EXIST 2022*](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&cad=rja&uact=8&ved=2ahUKEwij_P6v_dr7AhVsXqQEHVePCWIQFnoECA4QAQ&url=http%3A%2F%2Fjournal.sepln.org%2Fsepln%2Fojs%2Fojs%2Findex.php%2Fpln%2Farticle%2Fview%2F6443&usg=AOvVaw3lrB02zOxmsnIoeiyGQBqA), en los que se describe la motivación y los objetivos del problema del sexismo, los procesos de recolección de datos, normativa de la competición, evaluación de resultados y las propuestas de los participantes.

## 2. Estructura del notebook

1. Introducción al notebook
2. Estructura del notebook
3. Recopilación de datos
4. Muestreo de datos
5. Etiquetado de datos
6. Lectura y carga de datos
7. Análisis de características

## 3. Recopilación de datos

El proceso de recolección de datos fue orientado primeramente a establecer un **conjunto de expresiones populares y términos sexistas**, en español e inglés, que manifiestan una infravolaración hacia el rol de la mujer en la sociedad. Para ello se inspiraron en trabajos previamente realizados en el mismo área, cuentas y hashtags populares de redes sociales, etc. A continuación se resumen los métodos de recolección de datos diferenciados por la red social tomada como fuente de datos.

### 3.1. Twitter

La recogida de datos procedentes de esta plataforma social se elaboró en **dos períodos**, una primera fase de Diciembre de 2020 hasta Febrero de 2021 para componer la **primera versión** de un dataset de entrenamiento y otro de validación. Mientras que en la **segunda edición** de la competición el objetivo consistía en **ampliar el conjunto de datos de validación** y por ende se realizó una segunda fase de recopilación de tweets durante el mes de Enero de 2022. En sendos intervalos se aplicaron **técnicas de balanceado**, como la eliminación de documentos, para equilibrar el número de textos en inglés y español.

### 3.2. Gab

Únicamente se celebró **una etapa** de recolección de datos desde **2016 hasta 2019** resultando en millones de textos en inglés, mientras que apenas se obtuvieron dos mil documentos en español. En este caso no se aplicaron técnicas de balanceado entre idiomas por lo que el conjunto de datos procedente de esta red social está prácticamente **sesgado hacia el inglés** en su totalidad.

## 4. Muestreo de datos

En esta segunda fase se pretende valorar la calidad de las muestras recopiladas en función de las expresiones y términos sexistas establecidos que componen las bases de la población de textos con la que se pretende aboradar el problema. De nuevo se llevaron a cabo dos procedimientos diferentes dependiendo de la fuente de datos.

### 4.1. Twitter

Tras diversos análisis estadísticos pudieron confirmar que el conjunto de tweets recopilados se caracterizaban por una **distribución uniforme**, por lo que para **cada expresión y término** considerados, se asignó un **volumen de tweets representativos** con los que componer un dataset de entrenamiento y otro de validación.

### 4.2. Gab

En esta segunda fuente de datos los resultados estadísticos **no** pudieron determinar la presencia de una **distribución uniforme**, por lo que el proceso de selección de instancias aplicado fue diferente. 

1. En primer lugar se **eliminaron aquellos textos con información personal** de los autores, de modo que se redujese el sesgo introducido por los usuarios. 

2. A continuación se fijaron **intervalos temporales diferenciables** para seleccionar las muestras del conjunto de entrenamiento y de validación, decrementando así el sesgo temporal existente. 

3. Posteriormente se impuso una **primera restricción** en la que **cada usuario** debía estar ligado, al menos a **un texto**, aunque en el caso de documentos españoles esta cifra se elevó a dos publicaciones por usuario. 

4. Finalmente la **segunda restricción** se basó en la única aparición de **cada usuario en un conjunto de datos**: entrenamiento o validación, para evitar que una misma cuenta perteneciese a ambos.

## 5. Etiquetado de datos

En ambas ediciones de la competición datadas en 2021 y 2022 se conformaron varios **equipos de expertos en sexismo** con diversos grados de experiencia para etiquetar manualmente los textos recopilados. En los **primeros experimentos** los expertos disponían de una **menor experiencia**, mientras que en las **últimas iteraciones los más experimentados** terminaban de corregir las etiquetas del conjunto final *EXIST*.

Una de las principales diferencias existentes en este procedimiento de etiquetado entre las ediciones de 2021 y 2022 fue el uso de una **plataforma de ayuda al etiquetado de datos denominada AWS MTurk** *(Amazon Mechanical Turk)* que se empleó en combinación con un equipo de expertos utilizando un sistema de voto mayoritario para elegir la clase de cada instancia. Debido a que **AWS MTurk no es capaz de procesar emojis** estos fueron **convertidos en texto** reemplazándolos por sus correspondientes descripciones en lenguaje natural.

## 6. Lectura y carga de datos

En esta segunda sección se pretende **cargar los datasets de entrenamiento y validación** procedentes de los correspondientes ficheros situados en la carpeta *data*. Al tener un **formato TSV** se deben leer como tablas aunque posteriormente se trabaje con ellos en formato *dataframe*. 

Tal y como se puede comprobar en los siguientes resultados las dimensiones de sendos conjuntos de datos se detallan a continuación:

* Conjunto de entrenamiento: **6.977 muestras**.
* Conjunto de validación: **4.368 muestras**.

In [4]:
# Instalación y carga de las librerías necesarias
# 1. pandas: lectura y carga de datos tabulares como dataframes
%pip install pandas==1.5.2
import pandas as pd
# 2. collections: uso de estructuras de datos de tipo colección
from collections import Counter

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Lectura y carga de los datasets
training_df = pd.read_table("../data/EXIST2021_training.tsv")
testing_df = pd.read_table("../data/EXIST2021_test.tsv")

# Dimensiones de los datasets
print("Dimensiones del dataset de entrenamiento:", training_df.shape)
print("Dimensiones del dataset de validación:", testing_df.shape)

Dimensiones del dataset de entrenamiento: (6977, 7)
Dimensiones del dataset de validación: (4368, 7)


## 7. Análisis de características

Este primer análisis consiste en conocer el número de columnas, tipos de datos y significados con el objetivo de comprender los datos que se proporcionan sendos datasets. Según la información recopilada del artículo [*Overview of EXIST 2022*](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&cad=rja&uact=8&ved=2ahUKEwij_P6v_dr7AhVsXqQEHVePCWIQFnoECA4QAQ&url=http%3A%2F%2Fjournal.sepln.org%2Fsepln%2Fojs%2Fojs%2Findex.php%2Fpln%2Farticle%2Fview%2F6443&usg=AOvVaw3lrB02zOxmsnIoeiyGQBqA), a continuación se detalla cuál es la naturaleza de cada una de las columnas disponibles: 

* `test_case`: columna nominal que indica la edición de la competición a la que pertenece un registro.
* `id`: columna numérica que representa un identificador único para un registro.
* `source`: columna nominal que define la fuente de la que procede un texto, Twitter o Gab.
* `language`: columna nominal que define el idioma de un texto, inglés (*en*) o español (*es*).
* `text`: columna nominal con el contenido de un texto.
* `task1`: columna nominal que representa la variable a predecir que indica si un texto es sexista (*sexist*) o no (*non-sexist*).
* `task2`: columna nominal que representa la categoría sexista a la que pertenece un texto.

A continuación se calcula la frecuencia de los valores nominales de las columnas categóricas más relevantes para abordar el problema del sexismo y acerca de ellos se pueden obtener las siguientes conclusiones:

* En el conjunto de **entrenamiento** únicamente se encuentran textos procedentes de **Twitter**.

* Se confirma de nuevo el equilibrio entre los tweets redactados en español e inglés, con un aumento liviano en los textos españoles.

* Si bien las clases *sexist* y *non-sexist* se encuentran balanceadas, hay una **ligera diferencia incremental de textos no sexistas**.

* Finalmente se puede apreciar la existencia de un **mayor número de tweets de ideología y estereotipos** sexistas, los documentos **más difíciles de detectar** por modelos de Aprendizaje Automático debido a que el nivel de sexismo presentado es inferior al de otros textos, como los de violencia de género y sexual.


In [14]:
for column in ["source", "language", "task1", "task2"]:
    print(f"Frecuencia de valores nominales en la columna: {column}")
    column_frequencies = dict(Counter(list(training_df[column].values)))
    print(dict(sorted(column_frequencies.items(), key=lambda item: item[1], reverse=True)))

Frecuencia de valores nominales en la columna: source
{'twitter': 6977}
Frecuencia de valores nominales en la columna: language
{'es': 3541, 'en': 3436}
Frecuencia de valores nominales en la columna: task1
{'non-sexist': 3600, 'sexist': 3377}
Frecuencia de valores nominales en la columna: task2
{'non-sexist': 3600, 'ideological-inequality': 866, 'stereotyping-dominance': 809, 'misogyny-non-sexual-violence': 685, 'sexual-violence': 517, 'objectification': 500}


Si se replica el mismo análisis de frecuencias sobre el dataset de **validación** se pueden determinar los siguientes aspectos:

* A diferencia del conjunto de entrenamiento, en este sí que **aparecen textos procedentes de la red social Gab** aunque en una cifra sumamente **minoritaria** con respecto al número de tweets.

* Si bien el volumen de documentos en inglés y español continua siendo equilibrado, en este conjunto es el inglés el que cuenta con una mayor representatividad.

* Adicionalmente también se modifica el *ranking* de las categorías sexistas, siendo la misoginia una de las más voluminosas a diferencia del dataset de entrenamiento.

In [15]:
for column in ["source", "language", "task1", "task2"]:
    print(f"Frecuencia de categorías para la columna: {column}")
    column_frequencies = dict(Counter(list(testing_df[column].values)))
    print(dict(sorted(column_frequencies.items(), key=lambda item: item[1], reverse=True)))

Frecuencia de categorías para la columna: source
{'twitter': 3386, 'gab': 982}
Frecuencia de categorías para la columna: language
{'en': 2208, 'es': 2160}
Frecuencia de categorías para la columna: task1
{'sexist': 2281, 'non-sexist': 2087}
Frecuencia de categorías para la columna: task2
{'non-sexist': 2087, 'ideological-inequality': 621, 'misogyny-non-sexual-violence': 472, 'stereotyping-dominance': 464, 'sexual-violence': 400, 'objectification': 324}
